In [65]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

newnames_reco = {"aice_body-sharp":"DLR", 
                 "aice2_body-sharp":"DLR2", 
                 "first_body":"MBIR", 
                 "fbp_fc08":"FBP", 
                 "fbp2_fc08":"FBP2", 
                 "aidr3d_fc08":"HIR", 
                 "asir_standard":"HIR", 
                 "fbp_standard":"FBP",
                 "qr40f-q3_0.4":"Qr40f-Q3_0.4",
                 "qr40f-q3_0.6":"Qr40f-Q3_0.6",
                 "qr40f-q3_1":"Qr40f-Q3_1",
                 "br40f-q3_0.4":"Br40f-Q3_0.4",
                 "br40f-q3_0.6":"Br40f-Q3_0.6",
                 "br40f-q3_1":"Br40f-Q3_1"}

newnames_ct = {
    "CT4": "<b>Vendor 1",
    "GE": "<b>Vendor 2",
    "photon":"<b>Vendor 3",
}

newnames_network = {
    "3d_fullres_LiTS_151":"<b>3D",
    "3d_fullres_LiTS_151_res":"<b>3Dres",
}
size_filter = 4/3 * 3.14159265359 * (5/2)**3   # in cubic millimeters, 5mm sphere

dice_detection_threshold = 0.0
output_dir = '../../plots/3) deep learning performance eval/lesion_overall'


In [66]:
df_lesion_meta = pd.read_csv('results_csv/results_lesion_paper.csv')
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"volume_pred"] = None
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"dice"] = None
df_lesion_meta = df_lesion_meta[(df_lesion_meta["volume_pred"] > size_filter) | (df_lesion_meta["volume_pred"].isna()) ]
df_lesion_meta["detected"] = df_lesion_meta["dice"] > dice_detection_threshold
filter_lesion_props = (df_lesion_meta["C in HU"] < -10) | (df_lesion_meta["C in HU"].isna())
df_lesion_meta = df_lesion_meta[filter_lesion_props]
df_lesion_meta = df_lesion_meta[df_lesion_meta["kernel"] != "FC13"]

/var/folders/h7/vwbjm16917g8kt81x3my27d40000gn/T/ipykernel_21352/1010585138.py:1: DtypeWarning:

Columns (1,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.



In [67]:
df_lesion_meta["lesion_prop"] = df_lesion_meta.apply(lambda x: f"{str(2*int(x['R in mm'])).zfill(2)}mm/{int(x['C in HU'])}HU" if not np.isnan(x['R in mm']) else None , axis=1)
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco"].str.lower() + "_" + df_lesion_meta["kernel"].str.lower()
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco+kernel"].map(newnames_reco)
df_lesion_meta["network"] = df_lesion_meta["network"].map(newnames_network)
df_lesion_meta["ct"] = df_lesion_meta["ct"].map(newnames_ct)



In [68]:
#250322
df_lesion_meta = df_lesion_meta[df_lesion_meta["date"].isin([230915, 231205, 240306, 250727])]
df_lesion_meta["ct"] = df_lesion_meta.apply(lambda x: f"{x['ct']}_resliced" if x['date'] == 250727 else x['ct'], axis=1)
df_lesion_meta["ct"].unique()

array(['<b>Vendor 2_resliced', '<b>Vendor 1', '<b>Vendor 2'], dtype=object)

In [69]:
df_lesions = pd.read_csv('results_csv/results_lesions_paper.csv')
df_lesions["reco+kernel"] = df_lesions["reco"].str.lower() + "_" + df_lesions["kernel"].str.lower()
df_lesions["reco+kernel"] = df_lesions["reco+kernel"].map(newnames_reco)
df_lesions["network"] = df_lesions["network"].map(newnames_network)
df_lesions["ct"] = df_lesions["ct"].map(newnames_ct)
df_lesions = df_lesions[df_lesions["kernel"] != "FC13"]
df_lesions = df_lesions[df_lesions["date"].isin([230915, 231205, 240306, 250727])]
df_lesions["ct"] = df_lesions.apply(lambda x: f"{x['ct']}_resliced" if x['date'] == 250727 else x['ct'], axis=1)
df_lesions["ct"].unique()


array(['<b>Vendor 2_resliced', '<b>Vendor 2', '<b>Vendor 1'], dtype=object)

In [70]:
groupby_lesion_prop = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU"]
groupby_lesion_prop_rep = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "rep"]
groupby_lesion_prop_phantom_rep = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "phantom", "rep"]
groupby_lesion_all = ["network", "CTDIvol", "reco+kernel", "ct"]
groupby_lesion_all_phantom_rep = ["network", "CTDIvol", "reco+kernel", "ct", "phantom", "rep"]

In [71]:
df_lesion_meta_recall = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_meta_recall = df_lesion_meta_recall.groupby(groupby_lesion_all).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall.columns = groupby_lesion_all + ["recall", "num_detected", "num_total"]

df_lesion_meta["FP"] = df_lesion_meta["filename_gt"].isna() & df_lesion_meta["filename_pred"].notna() & (df_lesion_meta["px_numb_pred"] >= 0)
df_lesion_meta_precision = df_lesion_meta.groupby(groupby_lesion_all).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision.columns = groupby_lesion_all + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]

lesion_seg_summary = df_lesion_meta_recall.merge(df_lesion_meta_precision, on=groupby_lesion_all, how="left")
lesion_seg_summary["F1-score"] = 2 * (lesion_seg_summary["precision"] * lesion_seg_summary["recall"]) / (lesion_seg_summary["precision"] + lesion_seg_summary["recall"])
lesion_seg_summary["F1-score"] = lesion_seg_summary["F1-score"].fillna(0)

In [72]:
df_lesion_meta_recall_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_meta_recall_rep = df_lesion_meta_recall_rep.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_rep.columns = groupby_lesion_all + ["rep"] + ["recall", "num_detected", "num_total"]

df_lesion_meta_precision_rep = df_lesion_meta.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision_rep.columns = groupby_lesion_all + ["rep"] + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]

lesion_seg_summary_rep = df_lesion_meta_recall_rep.merge(df_lesion_meta_precision_rep, on=groupby_lesion_all + ["rep"], how="left")
lesion_seg_summary_rep["F1-score"] = 2 * (lesion_seg_summary_rep["precision"] * lesion_seg_summary_rep["recall"]) / (lesion_seg_summary_rep["precision"] + lesion_seg_summary_rep["recall"])
lesion_seg_summary_rep["F1-score"] = lesion_seg_summary_rep["F1-score"].fillna(0)



In [73]:
df_lesion_meta_recall_phantom_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()]
df_lesion_meta_recall_phantom_rep = df_lesion_meta_recall_phantom_rep.groupby(groupby_lesion_all_phantom_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_phantom_rep.columns = groupby_lesion_all_phantom_rep + ["recall", "num_detected", "num_total"]
df_lesion_meta_precision_phantom_rep = df_lesion_meta.groupby(groupby_lesion_all_phantom_rep).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"], "FP":"sum"}).reset_index()
df_lesion_meta_precision_phantom_rep.columns = groupby_lesion_all_phantom_rep + ["precision", "num_detected", "num_total", "dice_mean", "dice_std", "FP"]
lesion_seg_summary_phantom_rep = df_lesion_meta_recall_phantom_rep.merge(df_lesion_meta_precision_phantom_rep, on=groupby_lesion_all_phantom_rep, how="left")
lesion_seg_summary_phantom_rep["F1-score"] = 2 * (lesion_seg_summary_phantom_rep["precision"] * lesion_seg_summary_phantom_rep["recall"]) / (lesion_seg_summary_phantom_rep["precision"] + lesion_seg_summary_phantom_rep["recall"])


In [74]:
lesions_seg_summary = df_lesions.groupby(groupby_lesion_all + ["rep"]).agg({"dice":["mean", "std"]}).reset_index()
lesions_seg_summary.columns = groupby_lesion_all + ["rep", "dice_mean", "dice_std"]

In [75]:
lesion_seg_summary["CTDIvol"] = lesion_seg_summary["CTDIvol"].astype(float).round(5).astype(str)
lesion_seg_summary_rep["CTDIvol"] = lesion_seg_summary_rep["CTDIvol"].astype(float).round(5).astype(str)
lesions_seg_summary["CTDIvol"] = lesions_seg_summary["CTDIvol"].astype(float).round(5).astype(str)


In [76]:
# fig = px.bar(lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 1") & (lesion_seg_summary["network"].isin(["<b>3Dres", "<b>3D"]))],
#              x="CTDIvol", y="F1-score", 
#              color="reco+kernel", facet_row="ct", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], barmode="group", template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_canon.png")

In [77]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="reco+kernel", facet_row="ct", facet_col="network",
             range_y=[0,1], template="simple_white", 
             hover_data=["precision", "recall"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "ct": ["<b>Vendor 1", "<b>Vendor 2",],
                              "network": ["<b>3D", "<b>3Dres"]})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>", matches=None)
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=850,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_points_dicethreshold_{dice_detection_threshold}.png"))

In [78]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]
fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="recall",
             color="reco+kernel", facet_row="ct", facet_col="network",
             range_y=[0,1], template="simple_white", 
             hover_data=["precision", "F1-score"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D", "<b>3Dres"],
                              "ct": ["<b>Vendor 1", "<b>Vendor 2",],})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>", matches=None)
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=850,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_points_dicethreshold_{dice_detection_threshold}.png"))

In [79]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="precision",
             color="reco+kernel", facet_row="ct", facet_col="network",
             range_y=[0,1], template="simple_white", 
             hover_data=["recall", "F1-score"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D", "<b>3Dres"],
                              "ct": ["<b>Vendor 1", "<b>Vendor 2"]})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>Precision", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>", matches=None)
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=800,
    width=1100
    
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_precision_points_dicethreshold_{dice_detection_threshold}.png"))

In [80]:
# calc AUC for each curve F1 vs CTDIvol
from sklearn.metrics import auc

lesion_seg_summary_rep_auc = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_auc = lesion_seg_summary_rep_auc[lesion_seg_summary_rep_auc["ct"].isin(["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2_resliced"])]
lesion_seg_summary_rep_auc = lesion_seg_summary_rep_auc[lesion_seg_summary_rep_auc["CTDIvol"].astype(float) < 12]

aucs = []
for group, df in lesion_seg_summary_rep_auc.groupby(["ct", "network", "reco+kernel", "rep"]):
    #print(group)
    x = df["CTDIvol"]
    #print(x)
    x = x.astype(float)
    x = x/x.max()
    y = df["F1-score"]
    #print(y)
    aucs.append((group, auc(x, y)))

aucs
aucs_df = pd.DataFrame(aucs, columns=["group", "AUC"])
# split group into network and reco+kernel and rep and keep AUC column
aucs_df["ct"] = aucs_df["group"].apply(lambda x: x[0])
aucs_df["network"] = aucs_df["group"].apply(lambda x: x[1])
aucs_df["reco+kernel"] = aucs_df["group"].apply(lambda x: x[2])
aucs_df["rep"] = aucs_df["group"].apply(lambda x: x[3])
aucs_df = aucs_df.drop(columns=["group"])

aucs_df_mean = aucs_df.groupby(["ct","network", "reco+kernel"]).agg({"AUC":["mean", "std"]}).reset_index()
aucs_df_mean.columns = ["ct", "network", "reco+kernel", "AUC_mean", "AUC_std"]

In [81]:
aucs_df_mean_plot = aucs_df_mean[aucs_df_mean["network"].isin(["<b>3D", "<b>3Dres"])]
aucs_df_mean_plot = aucs_df_mean_plot[aucs_df_mean_plot["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]

fig = px.bar(aucs_df_mean_plot, x="reco+kernel", y="AUC_mean", error_y="AUC_std", color="reco+kernel", 
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_row="ct", facet_col="network",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D", "<b>3Dres"]},
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=500,
       width=600
       )
fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_dicethreshold_{dice_detection_threshold}.png"))


In [82]:
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.multitest import multipletests

reco_comparisons = []
networks = []
vendors = []
reco_1_list = []
reco_1_aucs = []
reco_2_list = []
reco_2_aucs = []
t_statistic = []
p_values = []
p_values_holm = []
p_values_othercorrection = []

for network in aucs_df["network"].unique():
    for ct in aucs_df["ct"].unique():
        p_values_temp = []
        reco_order = {"FBP": 0, "HIR": 1, "MBIR": 2, "DLR": 3}
        recos = aucs_df[aucs_df["network"] == network]["reco+kernel"].unique()
        recos = sorted(recos, key=lambda x: reco_order[x])
        for reco in recos:
            for reco_2 in recos[1:]:
                reco_comparison = f"{sorted([reco, reco_2])} for network {network} and ct {ct}"
                if reco != reco_2 and reco_comparison not in reco_comparisons:
                    reco_comparisons.append(reco_comparison)
                    aucs_reco = aucs_df[(aucs_df["network"] == network) & (aucs_df["reco+kernel"] == reco) & (aucs_df["ct"] == ct)]["AUC"]
                    aucs_reco_2 = aucs_df[(aucs_df["network"] == network) & (aucs_df["reco+kernel"] == reco_2) & (aucs_df["ct"] == ct)]["AUC"]
                    if len(aucs_reco) == 0 or len(aucs_reco_2) == 0:
                        continue    
                    else:
                        print(f"{ct} {network} {reco} vs {reco_2}")
                       # print(f" AUCs {aucs_reco.values} vs {aucs_reco_2.values}")
                    t, p = ttest_rel(aucs_reco, aucs_reco_2)
                    p_values_temp.append(p)
                    networks.append(network)
                    vendors.append(ct)
                    reco_1_list.append(reco)
                    reco_1_aucs.append(f"{aucs_reco.values.mean()}±{aucs_reco.values.std()}")
                    reco_2_list.append(reco_2)
                    reco_2_aucs.append(f"{aucs_reco_2.values.mean()}±{aucs_reco_2.values.std()}")
                    t_statistic.append(t)
        
        # replace p-values with nan 
        p_values_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_values_temp]
        p_values.extend(p_values_temp)

        # correct p-values for multiple comparisons
        reject, pvals_corrected, _, _ = multipletests(p_values_temp, method='holm')
        p_values_holm.extend(pvals_corrected)
        # you can add other correction methods if needed
        reject, pvals_corrected_other, _, _ = multipletests(p_values_temp, method='fdr_bh')
        p_values_othercorrection.extend(pvals_corrected_other)

df_stats_aucs_reco = pd.DataFrame({
    "network": networks,
    "ct": vendors,
    "reco_1": reco_1_list,
    "reco_2": reco_2_list,
    "t_statistic": t_statistic,
    "p_value": p_values,
    "p_value_holm": p_values_holm,
    "p_value_fdr_bh": p_values_othercorrection
})

df_stats_aucs_reco.to_csv(os.path.join(output_dir, f"lesion_seg_f1_score_auc_stats_dicethreshold_{dice_detection_threshold}.csv"), index=False)

<b>Vendor 1 <b>3D FBP vs HIR
<b>Vendor 1 <b>3D FBP vs MBIR
<b>Vendor 1 <b>3D FBP vs DLR
<b>Vendor 1 <b>3D HIR vs MBIR
<b>Vendor 1 <b>3D HIR vs DLR
<b>Vendor 1 <b>3D MBIR vs DLR
<b>Vendor 2 <b>3D FBP vs HIR
<b>Vendor 2_resliced <b>3D FBP vs HIR
<b>Vendor 1 <b>3Dres FBP vs HIR
<b>Vendor 1 <b>3Dres FBP vs MBIR
<b>Vendor 1 <b>3Dres FBP vs DLR
<b>Vendor 1 <b>3Dres HIR vs MBIR
<b>Vendor 1 <b>3Dres HIR vs DLR
<b>Vendor 1 <b>3Dres MBIR vs DLR
<b>Vendor 2 <b>3Dres FBP vs HIR
<b>Vendor 2_resliced <b>3Dres FBP vs HIR


In [83]:
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

# perform paired t-test for lesion detection all recos vs all recos
networks = []
vendors = []
reco_1_list = []
reco_2_list = []
dose_list = []
t_statistic = []
p_values = []
p_values_holm = []
p_values_othercorrection = []
reco_comparisons = []

for network in ["<b>3D", "<b>3Dres"]:
    for vendor in ["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2_resliced"]:
        reco_order = {"FBP": 0, "HIR": 1, "MBIR": 2, "DLR": 3}
        recos = lesion_seg_summary_rep[lesion_seg_summary_rep["ct"] == vendor]["reco+kernel"].unique()
        recos = sorted(recos, key=lambda x: reco_order[x])
        for reco_1 in recos:
            for reco_2 in recos:
                p_values_temp = []
                doses = lesion_seg_summary_rep[lesion_seg_summary_rep["ct"] == vendor]["CTDIvol"].unique()
                doses = sorted(doses, key=lambda x: float(x))
                for dose in doses:
                    reco_comparison = f"{sorted([reco_1, reco_2])} for {dose} {vendor} {network}"
                    if reco_1 == reco_2 or reco_comparison in reco_comparisons:
                        continue
                    #print(reco_comparison)

                    data_temp = lesion_seg_summary_rep[ (lesion_seg_summary_rep["ct"] == vendor) & (lesion_seg_summary_rep["CTDIvol"] == dose) & (lesion_seg_summary_rep["network"] == network)]
                    data_temp = data_temp.sort_values(by=["rep"])
                    data_reco_1 = data_temp[data_temp["reco+kernel"] == reco_1]["F1-score"]
                    data_reco_2 = data_temp[data_temp["reco+kernel"] == reco_2]["F1-score"]
                    #print(f"Vendor 1, CTDIvol {dose}: FBP: {data_FBP.values}, HIR: {data_HIR.values}")

                    result_stat = stats.ttest_rel(data_reco_1, data_reco_2)
                    #print(f"Vendor 1, CTDIvol {dose}: t-statistic: {result_stat.statistic}, p-value: {result_stat.pvalue} post Bonferroni: {min(result_stat.pvalue * correction_factor_reco , 1.0)}")

                    reco_comparisons.append(reco_comparison)
                    dose_list.append(dose)
                    t_statistic.append(result_stat.statistic)
                    p_values_temp.append(result_stat.pvalue)
                    networks.append(network)
                    vendors.append(vendor)
                    reco_1_list.append(reco_1)
                    reco_2_list.append(reco_2)

                if len(p_values_temp) == 0:
                    continue
                # set nan p_values to 1.0
                p_values_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_values_temp]
                print(len(p_values_temp))
                # apply bonferroni correction using stats.multitest.multipletests
                p_values += p_values_temp
                p_values_holm+= list(multipletests(p_values_temp, method="holm")[1])
                p_values_othercorrection += list(multipletests(p_values_temp, method="fdr_bh", alpha=0.05)[1])


data_frame_stats_reco_comp = pd.DataFrame({
    "network": networks,
    "vendor": vendors,
    "dose": dose_list,
    "reco_1": reco_1_list,
    "reco_2": reco_2_list,
    "t_statistic": t_statistic,
    "p_value": p_values,
    "p_value_holm": p_values_holm,
    "p_value_other_correction": p_values_othercorrection,
    "significant_holm": [pv < 0.05 for pv in p_values_holm],
    "significant_other_correction": [pv < 0.05 for pv in p_values_othercorrection],
})



data_frame_stats_reco_summary = data_frame_stats_reco_comp.groupby(["network", "vendor", "reco_1", "reco_2"]).agg({"significant_holm": "sum", "significant_other_correction": "sum", "dose": "count"}).reset_index()
doses_different_holm = data_frame_stats_reco_comp[data_frame_stats_reco_comp["significant_holm"] == True].groupby(["network", "vendor", "reco_1", "reco_2"]).agg({"dose": "unique"}).reset_index()
doses_different_other = data_frame_stats_reco_comp[data_frame_stats_reco_comp["significant_other_correction"] == True].groupby(["network", "vendor", "reco_1", "reco_2"]).agg({"dose": "unique"}).reset_index()
data_frame_stats_reco_summary = data_frame_stats_reco_summary.merge(doses_different_holm, on=["network", "vendor", "reco_1", "reco_2"], how="left", suffixes=("", "_different_holm"))
data_frame_stats_reco_summary = data_frame_stats_reco_summary.merge(doses_different_other, on=["network", "vendor", "reco_1", "reco_2"], how="left", suffixes=("", "_different_other"))

# save data_frame_stats and data_frame_stats_summary to one excel file with two sheets
with pd.ExcelWriter(os.path.join(output_dir, f"lesion_detection_statistical_analysis_recos_dicethreshold_{dice_detection_threshold}.xlsx")) as writer:
    data_frame_stats_reco_comp.to_excel(writer, sheet_name="detailed_results", index=False)
    data_frame_stats_reco_summary.to_excel(writer, sheet_name="summary_results", index=False)


34
34
34
34
34


/Users/genskeu/miniforge3/envs/val_tool/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.



34
40
40
34
34
34
34
34
34
40
40


In [84]:
from statsmodels.stats.anova import AnovaRM

data_rm = lesion_seg_summary_rep[
    (lesion_seg_summary_rep["ct"] == "<b>Vendor 1") &
    (lesion_seg_summary_rep["network"] == "<b>3D")
].copy()

# Create subject ID
data_rm["subject_id"] = data_rm["rep"].astype(str)
data_rm["reco_kernel"] = data_rm["reco+kernel"]
aovrm = AnovaRM(
    data_rm,
    depvar="F1-score",
    subject="subject_id",
    within=["reco_kernel", "CTDIvol"]
).fit()

aovrm.summary()

,F Value,Num DF,Den DF,Pr > F
reco_kernel,544.7881,3.0000,12.0000,0.0000
CTDIvol,202.9706,33.0000,132.0000,0.0000
reco_kernel:CTDIvol,19.0473,99.0000,396.0000,0.0000


In [85]:
# lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
# lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["reco+kernel"].isin(["HIR"])]
# fig = px.strip(lesion_seg_summary_rep_plot,
#              x="CTDIvol", y="precision",
#              color="rep", facet_row="ct", facet_col="network",
#              range_y=[0,1], template="simple_white", 
#              hover_data=["recall", "F1-score"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
#                               "network": ["<b>3D", "<b>3Dres"],
#                               "ct": ["<b>Vendor 1", "<b>Vendor 2"]})    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>Precision", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=800,
#     width=1100
    
# )
# fig.update_xaxes(matches=None)
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()


In [86]:
# lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep[(lesion_seg_summary_phantom_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_phantom_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
# lesion_seg_summary_phantom_rep_plot["CTDIvol"] = lesion_seg_summary_phantom_rep_plot["CTDIvol"].astype(float).round(1).astype(str)
# fig = px.strip(lesion_seg_summary_phantom_rep_plot,
#              x="CTDIvol", y="F1-score",
#              color="reco+kernel", facet_row="phantom", facet_col="network",
#              range_y=[0,1], template="simple_white", 
#              hover_data=["precision", "recall"],
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
#                               "network": ["<b>3D", "<b>3Dres"]})    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=800,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()
# fig.write_html(os.path.join(output_dir, f"f1_score_canon_points_dicethreshold_{dice_detection_threshold}_phantoms.html"))

In [87]:
# lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep[(lesion_seg_summary_phantom_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_phantom_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
# lesion_seg_summary_phantom_rep_plot["CTDIvol"] = lesion_seg_summary_phantom_rep_plot["CTDIvol"].astype(float).round(1).astype(str)
# fig = px.strip(lesion_seg_summary_phantom_rep_plot,
#              x="CTDIvol", y="F1-score",
#              color="reco+kernel", facet_row="phantom", facet_col="network",
#              range_y=[0,1], template="simple_white", 
#              hover_data=["precision", "recall"],
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
#                               "network": ["<b>3D", "<b>3Dres"]})    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=800,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()
# fig.write_html(os.path.join(output_dir, f"f1_score_ge_points_dicethreshold_{dice_detection_threshold}_phantoms.html"))

In [88]:

lesion_seg_summary_rep_data_plot = lesion_seg_summary_rep[lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"])]
lesion_seg_summary_rep_data_plot.drop(columns=["num_detected_x", "num_total_x", "num_detected_y", "num_total_y", "FP", "dice_mean", "dice_std"], inplace=True)
lesion_seg_summary_rep_data_plot.to_csv(os.path.join(output_dir, f"lesion_detection_summary_dicethreshold_{dice_detection_threshold}.csv"), index=False)
lesion_seg_summary_rep_data_plot.to_csv(f"../../../../../../promotion/daten_philipp/Figure 2/D/lesion_detection_summary_dicethreshold_{dice_detection_threshold}.csv", index=False)

In [89]:
# lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep[(lesion_seg_summary_phantom_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_phantom_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
# lesion_seg_summary_phantom_rep_plot["CTDIvol"] = lesion_seg_summary_phantom_rep_plot["CTDIvol"].astype(float).round(1).astype(str)

# fig = px.strip(lesion_seg_summary_phantom_rep_plot,
#              x="CTDIvol", y="F1-score",
#              color="phantom", facet_row="reco+kernel", facet_col="network",
#              range_y=[0,1], template="simple_white", 
#              hover_data=["precision", "recall"],
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
#                               "network": ["<b>3D", "<b>3Dres"]})    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=1100,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()

In [90]:
lesions_seg_summary_rep_plot = lesions_seg_summary[(lesions_seg_summary["network"].isin(["<b>3D", "<b>3Dres"]))]

fig = px.strip(lesions_seg_summary_rep_plot,
             x="CTDIvol", y="dice_mean",
             color="reco+kernel", facet_row="ct", facet_col="network",
             range_y=[0,1], template="simple_white", 
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D", "<b>3Dres"],
                              "ct": ["<b>Vendor 1", "<b>Vendor 2",],})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>DSC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>", matches=None)
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=850,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_dice_points.png"))

In [96]:
aucs_df_mean_plot = aucs_df_mean[aucs_df_mean["reco+kernel"].isin(["HIR"])]
aucs_df_mean_plot = aucs_df_mean_plot[aucs_df_mean_plot["network"].isin(["<b>3D", "<b>3Dres"])]
aucs_df_mean_plot = aucs_df_mean_plot[aucs_df_mean_plot["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]

fig = px.bar(aucs_df_mean_plot, x="reco+kernel", y="AUC_mean", error_y="AUC_std", color="network", 
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_row="ct",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D", "<b>3Dres"]}, barmode="group",
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=500,
       width=300
       
       )
fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_dicethreshold_{dice_detection_threshold}_networks.png"), width=300, height=500)

In [106]:
p_values_temp

[]

In [109]:
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.multitest import multipletests

comparisons = []
network_list_1 = []
network_list_2 = []
vendor_list = []
reco_list = []
t_statistic = []
p_values = []
p_values_holm = []
p_values_othercorrection = []

for ct in aucs_df["ct"].unique():
    reco_order = {"FBP": 0, "HIR": 1, "MBIR": 2, "DLR": 3}
    recos = aucs_df[aucs_df["network"] == network]["reco+kernel"].unique()
    recos = sorted(recos, key=lambda x: reco_order[x])
    for reco in recos:
        p_values_temp = []
        for network_1 in aucs_df["network"].unique():
            for network_2 in aucs_df["network"].unique():
                comparison = f"{sorted([network_1, network_2])} for reco {reco} and ct {ct}"
                if network_1 != network_2 and comparison not in comparisons:
                    comparisons.append(comparison)
                    aucs_network_1 = aucs_df[(aucs_df["network"] == network_1) & (aucs_df["reco+kernel"] == reco) & (aucs_df["ct"] == ct)]["AUC"]
                    aucs_network_2 = aucs_df[(aucs_df["network"] == network_2) & (aucs_df["reco+kernel"] == reco) & (aucs_df["ct"] == ct)]["AUC"]
                    if len(aucs_network_1) == 0 or len(aucs_network_2) == 0:
                        continue
                    else:
                        print(f"{ct} {reco} {network_1} vs {network_2}")
                       # print(f" AUCs {aucs_reco.values} vs {aucs_reco_2.values}")
                    t, p = ttest_rel(aucs_network_1, aucs_network_2)
                    p_values_temp.append(p)
                    network_list_1.append(network_1)
                    network_list_2.append(network_2)
                    vendor_list.append(ct)
                    reco_list.append(reco)
                    t_statistic.append(t)
        
        if len(p_values_temp) == 0:
            continue
        # replace p-values with nan 
        p_values_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_values_temp]
        p_values.extend(p_values_temp)

        # correct p-values for multiple comparisons
        reject, pvals_corrected, _, _ = multipletests(p_values_temp, method='holm')
        p_values_holm.extend(pvals_corrected)
        # you can add other correction methods if needed
        reject, pvals_corrected_other, _, _ = multipletests(p_values_temp, method='fdr_bh')
        p_values_othercorrection.extend(pvals_corrected_other)

df_stats_aucs_network = pd.DataFrame({
    "network_1": network_list_1,
    "network_2": network_list_2,
    "ct": vendor_list,
    "reco": reco_list,
    "t_statistic": t_statistic,
    "p_value": p_values,
    "p_value_holm": p_values_holm,
    "p_value_fdr_bh": p_values_othercorrection
})

df_stats_aucs_network.to_csv(os.path.join(output_dir, f"lesion_seg_f1_score_auc_stats_network_dicethreshold_{dice_detection_threshold}.csv"), index=False)

<b>Vendor 1 FBP <b>3D vs <b>3Dres
<b>Vendor 1 HIR <b>3D vs <b>3Dres
<b>Vendor 1 MBIR <b>3D vs <b>3Dres
<b>Vendor 1 DLR <b>3D vs <b>3Dres
<b>Vendor 2 FBP <b>3D vs <b>3Dres
<b>Vendor 2 HIR <b>3D vs <b>3Dres
<b>Vendor 2_resliced FBP <b>3D vs <b>3Dres
<b>Vendor 2_resliced HIR <b>3D vs <b>3Dres


In [110]:
df_stats_aucs_network

,network_1,network_2,ct,reco,t_statistic,p_value,p_value_holm,p_value_fdr_bh
0,<b>3D,<b>3Dres,<b>Vendor 1,FBP,-0.243380,0.819683,0.819683,0.819683
1,<b>3D,<b>3Dres,<b>Vendor 1,HIR,-7.941882,0.001361,0.001361,0.001361
2,<b>3D,<b>3Dres,<b>Vendor 1,MBIR,-6.725411,0.002546,0.002546,0.002546
3,<b>3D,<b>3Dres,<b>Vendor 1,DLR,-29.250354,0.000008,0.000008,0.000008
4,<b>3D,<b>3Dres,<b>Vendor 2,FBP,29.200948,0.000008,0.000008,0.000008
5,<b>3D,<b>3Dres,<b>Vendor 2,HIR,34.382916,0.000004,0.000004,0.000004
6,<b>3D,<b>3Dres,<b>Vendor 2_resliced,FBP,19.731193,0.000039,0.000039,0.000039
7,<b>3D,<b>3Dres,<b>Vendor 2_resliced,HIR,24.461308,0.000017,0.000017,0.000017


In [115]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"]) ) & (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"])) & (lesion_seg_summary_rep["reco+kernel"].isin(["HIR"]))]



fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="network", facet_row="ct",
             range_y=[0,1], template="simple_white", 
             hover_data=["precision", "recall"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D", "<b>3Dres"],
                              "ct": ["<b>Vendor 1", "<b>Vendor 2"]})    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=800,
    width=600
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.update_xaxes(matches=None)

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_points_dicethreshold_{dice_detection_threshold}_hir.png"), width=600, height=800)
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_points_dicethreshold_{dice_detection_threshold}.png"))

In [93]:
import scipy.stats as stats

# perform paired t-test for lesion detection only HIR and 3D vs 3Dres across all doses and vendors
network_1 = []
network_2 = []
vendors = []
dose_list = []
t_statistic = []
p_value = []
p_value_holm = []
p_value_other = []

for vendor in ["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2_resliced"]:
    doses = lesion_seg_summary_rep[lesion_seg_summary_rep["ct"] == vendor]["CTDIvol"].unique()
    doses = sorted(doses, key=lambda x: float(x))
    p_value_temp = []
    for dose in doses:
        data_temp = lesion_seg_summary_rep[ (lesion_seg_summary_rep["ct"] == vendor) & (lesion_seg_summary_rep["CTDIvol"] == dose)]
        data_1 = data_temp[(data_temp["reco+kernel"] == "HIR") & (data_temp["network"] == "<b>3D")]["F1-score"]
        data_2 = data_temp[(data_temp["reco+kernel"] == "HIR") & (data_temp["network"] == "<b>3Dres")]["F1-score"]
        #print(f"Vendor 1, CTDIvol {dose}: FBP: {data_FBP.values}, HIR: {data_HIR.values}")

        result_stat = stats.ttest_rel(data_1, data_2)

        reco_comparisons.append(reco_comparison)
        dose_list.append(dose)
        t_statistic.append(result_stat.statistic)
        p_value_temp.append(result_stat.pvalue)
        network_1.append("<b>3D")
        network_2.append("<b>3Dres")
        vendors.append(vendor)
    
    # apply holm correction
    p_value_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_value_temp]
    p_value += p_value_temp
    p_value_holm += list(multipletests(p_value_temp, method="holm")[1])
    p_value_other += list(multipletests(p_value_temp, method="fdr_bh", alpha=0.05)[1])

data_frame_stats_network_comp = pd.DataFrame({
    "network_1": network_1,
    "network_2": network_2,
    "vendor": vendors,
    "dose": dose_list,
    "t_statistic": t_statistic,
    "p_value": p_value,
    "p_value_holm": p_value_holm,
    "significant_holm": [p < 0.05 for p in p_value_holm],
    "p_value_other": p_value_other,
    "significant_other": [p < 0.05 for p in p_value_other],
})

data_frame_stats_network_comp_summary = data_frame_stats_network_comp.groupby(["network_1", "network_2", "vendor"]).agg({"significant_holm": "sum", "significant_other": "sum", "dose": "count"}).reset_index()
doses_different_holm = data_frame_stats_network_comp[data_frame_stats_network_comp["significant_holm"] == True].groupby(["network_1", "network_2", "vendor"]).agg({"dose": "unique"}).reset_index()
data_frame_stats_network_comp_summary = data_frame_stats_network_comp_summary.merge(doses_different_holm, on=["network_1", "network_2", "vendor"], how="left", suffixes=("", "_different"))
doses_different_other = data_frame_stats_network_comp[data_frame_stats_network_comp["significant_other"] == True].groupby(["network_1", "network_2", "vendor"]).agg({"dose": "unique"}).reset_index()
data_frame_stats_network_comp_summary = data_frame_stats_network_comp_summary.merge(doses_different_other, on=["network_1", "network_2", "vendor"], how="left", suffixes=("", "_different_other"))

# save data_frame_stats and data_frame_stats_summary to one excel file with two sheets
with pd.ExcelWriter(os.path.join(output_dir, f"lesion_detection_statistical_analysis_network_comparison_dicethreshold_{dice_detection_threshold}.xlsx")) as writer:
    data_frame_stats_network_comp.to_excel(writer, sheet_name="detailed_results", index=False)
    data_frame_stats_network_comp_summary.to_excel(writer, sheet_name="summary_results", index=False)

In [94]:
aucs_df_mean_plot = aucs_df_mean[aucs_df_mean["reco+kernel"].isin([ "HIR"]) ]
aucs_df_mean_plot = aucs_df_mean_plot[aucs_df_mean_plot["network"].isin(["<b>3D", "<b>3Dres"])]
aucs_df_mean_plot = aucs_df_mean_plot[aucs_df_mean_plot["ct"].isin(["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2_resliced"])]

fig = px.bar(aucs_df_mean_plot, x="reco+kernel", y="AUC_mean", error_y="AUC_std", color="ct", 
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_col="network",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D", "<b>3Dres"]}, barmode="group",
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=400,
       width=800
       
       )
fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_dicethreshold_{dice_detection_threshold}_vendors.png"), width=800, height=400)


In [95]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"].isin(["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2_resliced"]) ) & (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"])) & 
(lesion_seg_summary_rep["reco+kernel"].isin(["HIR"]))]
lesion_seg_summary_rep_plot["CTDIvol"] = lesion_seg_summary_rep_plot["CTDIvol"].astype(float).round(5)
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["CTDIvol"] <= 11 ]

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="ct", facet_col="network",
             range_y=[0,1], template="simple_white", 
             hover_data=["precision", "recall"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D", "<b>3Dres"],
                                "ct": ["<b>Vendor 1", "<b>Vendor 2"]})  

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_points_dicethreshold_{dice_detection_threshold}_vendors.png"), width=1100, height=500)
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_points_dicethreshold_{dice_detection_threshold}.png"))

/var/folders/h7/vwbjm16917g8kt81x3my27d40000gn/T/ipykernel_21352/3638834237.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [349]:
# interpolate F1-scores for vendor 2, reco HIR, both networks to match the CTDIvol values of vendor 1, for each rep
from scipy.interpolate import interp1d
lesion_seg_summary_rep_vendor1 = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep["reco+kernel"] == "HIR")]
lesion_seg_summary_rep_vendor2 = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_rep["reco+kernel"] == "HIR")]
networks = []
vendor = [] 
rep_list = []
interpolated_f1_list = []
interpolated_recall_list = []
interpolated_precision_list = []
ctdi_list = []
reco_list = []
for rep in lesion_seg_summary_rep_vendor2["rep"].unique():
    for network in ["<b>3D", "<b>3Dres"]:
        ctdivol_vendor1 = lesion_seg_summary_rep_vendor1[(lesion_seg_summary_rep_vendor1["network"] == network) & (lesion_seg_summary_rep_vendor1["rep"] == rep)]["CTDIvol"].astype(float).values
        f = interp1d(lesion_seg_summary_rep_vendor2[(lesion_seg_summary_rep_vendor2["network"] == network) & (lesion_seg_summary_rep_vendor2["rep"] == rep)]["CTDIvol"].astype(float).values,
                     lesion_seg_summary_rep_vendor2[(lesion_seg_summary_rep_vendor2["network"] == network) & (lesion_seg_summary_rep_vendor2["rep"] == rep)]["F1-score"].values,
                     kind="linear", fill_value="extrapolate")
        interpolated_f1 = f(ctdivol_vendor1)
        f_recall = interp1d(lesion_seg_summary_rep_vendor2[(lesion_seg_summary_rep_vendor2["network"] == network) & (lesion_seg_summary_rep_vendor2["rep"] == rep)]["CTDIvol"].astype(float).values,
                     lesion_seg_summary_rep_vendor2[(lesion_seg_summary_rep_vendor2["network"] == network) & (lesion_seg_summary_rep_vendor2["rep"] == rep)]["recall"].values,
                     kind="linear", fill_value="extrapolate")
        interpolated_recall = f_recall(ctdivol_vendor1)
        f_precision = interp1d(lesion_seg_summary_rep_vendor2[(lesion_seg_summary_rep_vendor2["network"] == network) & (lesion_seg_summary_rep_vendor2["rep"] == rep)]["CTDIvol"].astype(float).values,
                     lesion_seg_summary_rep_vendor2[(lesion_seg_summary_rep_vendor2["network"] == network) & (lesion_seg_summary_rep_vendor2["rep"] == rep)]["precision"].values,
                     kind="linear", fill_value="extrapolate")
        interpolated_precision = f_precision(ctdivol_vendor1)
        for i in range(len(ctdivol_vendor1)):
            networks.append(network)
            vendor.append("<b>Vendor 2 interpolated")
            rep_list.append(rep)
            interpolated_f1_list.append(interpolated_f1[i])
            interpolated_recall_list.append(interpolated_recall[i])
            interpolated_precision_list.append(interpolated_precision[i])
            ctdi_list.append(ctdivol_vendor1[i])
            reco_list.append("HIR")

lesion_seg_summary_rep_vendor2_interpolated = pd.DataFrame({
    "network": networks,
    "ct": vendor,
    "rep": rep_list,
    "F1-score": interpolated_f1_list,
    "recall": interpolated_recall_list,
    "precision": interpolated_precision_list,
    "CTDIvol": ctdi_list,
    "reco+kernel": reco_list
})

lesion_seg_summary_rep_interpolated = pd.concat([lesion_seg_summary_rep, lesion_seg_summary_rep_vendor2_interpolated], ignore_index=True)


In [350]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_interpolated[(lesion_seg_summary_rep_interpolated["ct"].isin(["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2 interpolated"]) ) & (lesion_seg_summary_rep_interpolated["network"].isin(["<b>3D", "<b>3Dres"])) & (lesion_seg_summary_rep_interpolated["reco+kernel"].isin(["HIR"]))]
lesion_seg_summary_rep_plot["CTDIvol"] = lesion_seg_summary_rep_plot["CTDIvol"].astype(float).round(3)

fig = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="ct", facet_col="network",
             range_y=[0,1], template="simple_white", 
             hover_data=["precision", "recall"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D", "<b>3Dres"],
                                "ct": ["<b>Vendor 1", "<b>Vendor 2", "<b>Vendor 2 interpolated"]})  

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b>F1-score", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_canon_points_dicethreshold_{dice_detection_threshold}.png"))

/var/folders/h7/vwbjm16917g8kt81x3my27d40000gn/T/ipykernel_9646/2258905897.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [351]:
data_temp

,network,CTDIvol,reco+kernel,ct,rep,recall,num_detected_x,num_total_x,precision,num_detected_y,num_total_y,dice_mean,dice_std,FP,F1-score
1070,<b>3D,26.12,FBP,<b>Vendor 2,1,0.791667,57,72,0.791667,57,72,0.760012,0.169661,0,0.791667
1071,<b>3D,26.12,FBP,<b>Vendor 2,2,0.805556,58,72,0.805556,58,72,0.744379,0.219409,0,0.805556
1072,<b>3D,26.12,FBP,<b>Vendor 2,3,0.805556,58,72,0.805556,58,72,0.757555,0.188317,0,0.805556
1073,<b>3D,26.12,FBP,<b>Vendor 2,4,0.791667,57,72,0.791667,57,72,0.768693,0.162610,0,0.791667
1074,<b>3D,26.12,FBP,<b>Vendor 2,5,0.791667,57,72,0.791667,57,72,0.764351,0.173854,0,0.791667
1075,<b>3D,26.12,HIR,<b>Vendor 2,1,0.791667,57,72,0.791667,57,72,0.764738,0.166799,0,0.791667
1076,<b>3D,26.12,HIR,<b>Vendor 2,2,0.805556,58,72,0.805556,58,72,0.752420,0.206205,0,0.805556
1077,<b>3D,26.12,HIR,<b>Vendor 2,3,0.805556,58,72,0.805556,58,72,0.759754,0.191421,0,0.805556
1078,<b>3D,26.12,HIR,<b>Vendor 2,4,0.791667,57,72,0.791667,57,72,0.775789,0.150039,0,0.791667
1079,<b>3D,26.12,HIR,<b>Vendor 2,5,0.791667,57,72,0.791667,57,72,0.768136,0.169486,0,0.791667


In [352]:
# statistical test between vendors for HIR and both networks at each dose level only HIR
networks = []
vendors = []
dose_list = []
t_statistic = []
p_values = []
p_values_holm = []
p_values_othercorrection = []

lesion_seg_summary_rep_interpolated["CTDIvol"] = lesion_seg_summary_rep_interpolated["CTDIvol"].astype(float).round(5)
for network in ["<b>3D", "<b>3Dres"]:
    doses = lesion_seg_summary_rep_interpolated[(lesion_seg_summary_rep_interpolated["network"] == network) & (lesion_seg_summary_rep_interpolated["ct"] == "<b>Vendor 1")]["CTDIvol"].unique()
    doses = sorted(doses, key=lambda x: float(x))
    p_values_temp = []
    for dose in doses:
        data_temp = lesion_seg_summary_rep_interpolated[ (lesion_seg_summary_rep_interpolated["network"] == network) & (lesion_seg_summary_rep_interpolated["CTDIvol"] == dose)]
        data_1 = data_temp[(data_temp["reco+kernel"] == "HIR") & (data_temp["ct"] == "<b>Vendor 1")]["F1-score"]
        data_2 = data_temp[(data_temp["reco+kernel"] == "HIR") & (data_temp["ct"] == "<b>Vendor 2 interpolated")]["F1-score"]
        result_stat = stats.ttest_rel(data_1, data_2)
        networks.append(network)
        vendors.append("Vendor 1 vs Vendor 2 interpolated")
        dose_list.append(dose)
        t_statistic.append(result_stat.statistic)
        p_values_temp.append(result_stat.pvalue)

    # fix nan p-values
    p_values_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_values_temp]
    p_values += p_values_temp  
    p_values_holm += list(multipletests(p_values_temp, method="holm")[1])
    p_values_othercorrection += list(multipletests(p_values_temp, method="fdr_bh", alpha=0.05)[1])

data_frame_stats_vendor_comp = pd.DataFrame({
    "network": networks,
    "vendor": vendors,
    "dose": dose_list,
    "t_statistic": t_statistic,
    "p_value": p_values,
    "p_value_holm": p_values_holm,
    "significant_holm": [pv < 0.05 for pv in p_values_holm],
    "p_value_other_correction": p_values_othercorrection,
    "significant_other_correction": [pv < 0.05 for pv in p_values_othercorrection],
})

data_frame_stats_vendor_comp_summary = data_frame_stats_vendor_comp.groupby(["network", "vendor"]).agg({"significant_holm": "sum", "significant_other_correction": "sum", "dose": "count"}).reset_index()
doses_different_holm = data_frame_stats_vendor_comp[data_frame_stats_vendor_comp["significant_holm"] == True].groupby(["network", "vendor"]).agg({"dose": "unique"}).reset_index()
data_frame_stats_vendor_comp_summary = data_frame_stats_vendor_comp_summary.merge(doses_different_holm, on=["network", "vendor"], how="left", suffixes=("", "_different"))
doses_different_other = data_frame_stats_vendor_comp[data_frame_stats_vendor_comp["significant_other_correction"] == True].groupby(["network", "vendor"]).agg({"dose": "unique"}).reset_index()
data_frame_stats_vendor_comp_summary = data_frame_stats_vendor_comp_summary.merge(doses_different_other, on=["network", "vendor"], how="left", suffixes=("", "_different_other"))
# save data_frame_stats and data_frame_stats_summary to one excel file with two sheets
with pd.ExcelWriter(os.path.join(output_dir, f"lesion_detection_statistical_analysis_vendor_comparison_hir_dicethreshold_{dice_detection_threshold}.xlsx")) as writer:
    data_frame_stats_vendor_comp.to_excel(writer, sheet_name="detailed_results", index=False)
    data_frame_stats_vendor_comp_summary.to_excel(writer, sheet_name="summary_results", index=False)

In [353]:
# lesion_seg_summary_rep_mean = lesion_seg_summary_rep.groupby(groupby_lesion_all).agg({"recall": ["mean", "std"], "precision": ["mean", "std"], "F1-score": ["mean", "std"]}).reset_index()
# lesion_seg_summary_rep_mean.columns = groupby_lesion_all + ["recall_mean", "recall_std", "precision_mean", "precision_std", "F1-score_mean", "F1-score_std"]
# doses = lesion_seg_summary_rep_mean[lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 1"]["CTDIvol"].unique()
# doses = sorted(doses, key=lambda x: float(x))
# fig = px.scatter(lesion_seg_summary_rep_mean[(lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 1") & 
#                                            (lesion_seg_summary_rep_mean["network"].isin(["<b>3D", "<b>3Dres"]))],
#              x="CTDIvol", y="recall_mean", error_y="recall_std",
#              color="reco+kernel", facet_row="ct", facet_col="network",
#              title="F1-score lesion detection", range_y=[0,1], template="simple_white", 
#              hover_data=["precision_mean", "recall_mean"],
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], "CTDIvol": doses },)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_canon_lines.png")

In [354]:
# lesion_seg_summary_rep_variance = lesion_seg_summary_rep.groupby(groupby_lesion_all).agg({"F1-score":["mean", "std"]}).reset_index()
# lesion_seg_summary_rep_variance.columns = groupby_lesion_all + ["F1-score_mean", "F1-score_std"]

# fig = px.bar(lesion_seg_summary_rep_variance[(lesion_seg_summary_rep_variance["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep_variance["network"].isin(["<b>3D", "<b>3Dres"]))],
#              x="CTDIvol", y="F1-score_std",
#              color="reco+kernel", facet_row="ct", facet_col="network",
#              title="F1-score lesion detection", range_y=[0,.1], template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],}, barmode="group")    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()
# #fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_canon_points.png")

In [355]:
# lesions_seg_summary_filtered = lesions_seg_summary[(lesions_seg_summary["ct"] == "<b>Vendor 1") & (lesions_seg_summary["network"].isin(["<b>3D", "<b>3Dres"]))]
# fig = px.bar(lesions_seg_summary_filtered,
#              x="CTDIvol", y="dice_std",
#              color="reco+kernel", facet_col="network",
#              title="Dice for lesion segmentation ov", range_y=[0,0.1], template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},barmode="group")    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>Dice", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_dice_score_canon_points.png")

In [356]:
fig = px.strip(lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))],
             x="CTDIvol", y="FP",
             color="reco+kernel", facet_row="ct", facet_col="network",
             title="Number of False Positives for lesion segmentation", template="simple_white", 
             hover_data=["precision", "recall"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                              "network": ["<b>3D", "<b>3Dres"]},)    

fig.update_xaxes(title_text="<b>CTDIvol", row=1)
fig.update_yaxes(title_text="<b># FP", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>", matches=None)
fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=850,
    width=1100
    
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

#fig.add_shape(type='rect', x0=4, x1=16, y0=-5, y1=60, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
#fig.add_shape(type='rect', x0=4, x1=16, y0=-5, y1=60, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_FP_points_dicethreshold_{dice_detection_threshold}.png"))


In [357]:
# fig = px.bar(lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 2") & (lesion_seg_summary["network"].isin(["<b>3Dres", "<b>3D"]))],
#               x="CTDIvol", y="F1-score", 
#              color="reco+kernel", facet_row="ct", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], barmode="group", template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR",],},)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)

# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
# )
# fig = px.bar(lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 2") & (lesion_seg_summary["network"].isin(["<b>3Dres", "<b>3D"]))],
#               x="CTDIvol", y="F1-score", 
#              color="reco+kernel", facet_row="ct", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], barmode="group", template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR",],},)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)

# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_ge.png")

In [358]:
# lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep[(lesion_seg_summary_phantom_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_phantom_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
# lesion_seg_summary_phantom_rep_plot["CTDIvol"] = lesion_seg_summary_phantom_rep_plot["CTDIvol"].astype(float).round(1).astype(str)

# fig = px.strip(lesion_seg_summary_phantom_rep_plot,
#              x="CTDIvol", y="F1-score",
#              color="phantom", facet_row="reco+kernel", facet_col="network",
#              range_y=[0,1], template="simple_white", 
#              hover_data=["precision", "recall"],
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR"],
#                               "network": ["<b>3D", "<b>3Dres"]})    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=850,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()

In [359]:
# lesion_seg_summary_rep_mean = lesion_seg_summary_rep.groupby(groupby_lesion_all).agg({"recall": ["mean", "std"], "precision": ["mean", "std"], "F1-score": ["mean", "std"]}).reset_index()
# lesion_seg_summary_rep_mean.columns = groupby_lesion_all + ["recall_mean", "recall_std", "precision_mean", "precision_std", "F1-score_mean", "F1-score_std"]
# doses = lesion_seg_summary_rep_mean[lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 2"]["CTDIvol"].unique()
# doses = sorted(doses, key=lambda x: float(x))
# fig = px.scatter(lesion_seg_summary_rep_mean[(lesion_seg_summary_rep_mean["ct"] == "<b>Vendor 2") & 
#                                            (lesion_seg_summary_rep_mean["network"].isin(["<b>3D", "<b>3Dres"]))],
#              x="CTDIvol", y="recall_mean", error_y="recall_std",
#              color="reco+kernel", facet_row="ct", facet_col="network",
#              title="F1-score lesion detection", range_y=[0,1], template="simple_white", 
#              hover_data=["precision_mean", "recall_mean"],
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], "CTDIvol": doses },)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
    
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# #fig.add_shape(type='rect', x0=4, x1=18, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=1)
# #fig.add_shape(type='rect', x0=4, x1=18, y0=0, y1=0.75, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=1, col=2)

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_ge_lines.png")

In [360]:
# lesion_seg_summary_comp = lesion_seg_summary[(lesion_seg_summary["reco+kernel"].isin(["HIR"]))]
# lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["reco+kernel"].isin(["HIR", "1"]))]
# lesion_seg_summary_comp["CTDIvol"] = lesion_seg_summary_comp["CTDIvol"].astype(float).round(2)
# cdtivalues = lesion_seg_summary_comp["CTDIvol"].unique()
# fig = px.line(lesion_seg_summary_comp[(lesion_seg_summary["network"].isin(["<b>3D", "<b>3Dres"]))],
#               x="CTDIvol", y="F1-score", 
#              color="ct", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR",], "CTDIvol": cdtivalues},)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)

# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_scanner_compare.png")

In [361]:
# import scipy.stats as stats

# # perform t-test fpr lesion detection vendor 1
# lesion_seg_summary_comp_vendor_1 = lesion_seg_summary_rep[lesion_seg_summary_rep["ct"] == "<b>Vendor 1"]
# lesion_seg_summary_comp_vendor_1 = lesion_seg_summary_comp_vendor_1[lesion_seg_summary_comp_vendor_1["network"].isin(["<b>3D", "<b>3Dres"])]
# lesion_seg_summary_comp_vendor_1 = lesion_seg_summary_comp_vendor_1[lesion_seg_summary_comp_vendor_1["reco+kernel"].isin(["HIR"])]
# doses = lesion_seg_summary_comp_vendor_1["CTDIvol"].unique()
# for dose in doses:
#     data_temp = lesion_seg_summary_comp_vendor_1[lesion_seg_summary_comp_vendor_1["CTDIvol"] == dose]
#     data_2d = data_temp[data_temp["network"] == "<b>3Dres"]["F1-score"]
#     data_3d = data_temp[data_temp["network"] == "<b>3D"]["F1-score"]


#     t_stat, p_val = stats.ttest_ind(data_2d, data_3d, )
#     print(f"Vendor 1, CTDIvol {dose}: t-statistic: {t_stat}, p-value: {p_val}, p-value_adj: {p_val*len(doses) } ")

In [362]:
# import scipy.stats as stats

# # perform t-test fpr lesion detection vendor 2
# lesion_seg_summary_comp_vendor_1 = lesion_seg_summary_rep[lesion_seg_summary_rep["ct"] == "<b>Vendor 2"]
# lesion_seg_summary_comp_vendor_1 = lesion_seg_summary_comp_vendor_1[lesion_seg_summary_comp_vendor_1["network"].isin(["<b>3D", "<b>3Dres"])]
# lesion_seg_summary_comp_vendor_1 = lesion_seg_summary_comp_vendor_1[lesion_seg_summary_comp_vendor_1["reco+kernel"].isin(["HIR"])]
# doses = lesion_seg_summary_comp_vendor_1["CTDIvol"].unique()
# for dose in doses:
#     data_temp = lesion_seg_summary_comp_vendor_1[lesion_seg_summary_comp_vendor_1["CTDIvol"] == dose]
#     data_2d = data_temp[data_temp["network"] == "<b>3Dres"]["F1-score"]
#     data_3d = data_temp[data_temp["network"] == "<b>3D"]["F1-score"]


#     t_stat, p_val = stats.ttest_ind(data_2d, data_3d, )
#     print(f"Vendor 1, CTDIvol {dose}: t-statistic: {t_stat}, p-value: {p_val}, p-value_adj: {p_val*len(doses) } ")

In [363]:
# lesion_seg_summary["kernel"] = lesion_seg_summary["reco+kernel"].apply(lambda x: x.split("_")[-1])
# lesion_seg_summary["reco"] = lesion_seg_summary["reco+kernel"].apply(lambda x: x.split("_")[0])
# lesion_seg_summary["CTDIvol"] = lesion_seg_summary["CTDIvol"].astype(float)
# lesion_seg_summary = lesion_seg_summary.sort_values(["CTDIvol", "network", "kernel",])
# lesion_seg_summary["CTDIvol"] = lesion_seg_summary["CTDIvol"].astype(str)
# lesion_seg_summary["CTDIvol"][lesion_seg_summary["CTDIvol"] == "6.2"] = "6.3"
# fig = px.line(lesion_seg_summary[(lesion_seg_summary["ct"] == "<b>Vendor 3") & (lesion_seg_summary["network"].isin(["<b>3Dres", "<b>3D"]))],
#               x="CTDIvol", y="F1-score", 
#              color="reco", facet_row="kernel", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR",],},)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)

# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=1000,
#     width=1100
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_photon.png")

In [364]:
# lesion_seg_summary_comp = lesion_seg_summary[(lesion_seg_summary["reco"].isin(["HIR", "Br40f-Q3"]))]
# lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["kernel"].isin(["HIR", "1"]))]
# cdtivalues = lesion_seg_summary_comp["CTDIvol"].unique()
# fig = px.line(lesion_seg_summary_comp[(lesion_seg_summary["network"].isin(["<b>3Dres", "<b>3D"]))],
#               x="CTDIvol", y="F1-score", 
#              color="ct", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR",], "CTDIvol": cdtivalues},)    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1)
# fig.update_yaxes(title_text="<b>F1-score", col=1)

# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_scanner_compare.png")

In [365]:
# lesion_seg_summary_comp = lesion_seg_summary[(lesion_seg_summary["reco"].isin(["HIR", "Br40f-Q3"]))]
# lesion_seg_summary_comp = lesion_seg_summary_comp[(lesion_seg_summary_comp["kernel"].isin(["HIR", "1"]))]
# lesion_seg_summary_comp["CTDIvol"] = lesion_seg_summary_comp["CTDIvol"].astype(float).round(1)
# #cdtivalues = lesion_seg_summary_comp["CTDIvol"].unique()
# fig = px.line(lesion_seg_summary_comp[(lesion_seg_summary["network"].isin(["<b>3Dres", "<b>3D"]))],
#               x="CTDIvol", y="F1-score", log_x=True,
#              color="ct", facet_col="network", 
#              title="F1-score lesion detection", range_y=[0,1], template="simple_white", 
#              color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#              category_orders={"reco+kernel": ["FBP", "HIR",],}, 
#              )    

# fig.update_xaxes(title_text="<b>CTDIvol", row=1, tickvals = [0,.5,1,2,4,8,16])
# fig.update_yaxes(title_text="<b>F1-score", col=1)

# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1100
# )

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.show()
# #fig.write_image("plots/3) deep learning performance eval/overall/lesion_seg_f1_score_scanner_compare.png")